# Web Scraping

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [2]:
def custom_strptime_detik(date_str):
    months = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'Mei': '05', 'Jun': '06', 'Jul': '07', 'Agu': '08', 'Ags': '08', 'Sep': '09', 'Okt': '10', 'Nov': '11', 'Des': '12'
    }
    date_parts = date_str.split()
    return datetime.strptime(f"{date_parts[0]} {months[date_parts[1]]} {date_parts[2]} {date_parts[3]}", '%d %m %Y %H:%M')

#### Detik Headlines
Pada halaman detik, konten yang dianalisa adalah summary pada halaman headlines

In [3]:
def scrape_detik_headlines(query, total_pages=4):
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # Buat list untuk menyimpan data
    data = []

    base_url = f"https://www.detik.com/search/searchall?query={query}&sortby=time&page={{}}"

    for page in range(1, total_pages + 1):
        URL = base_url.format(page)
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "lxml")
        articles = soup.find_all('article')

        for article in articles:
            link = article.find('a')['href']
            judul = article.find('h2', attrs={'class': 'title'}).string.strip()
            date = article.find('span', attrs={'class': 'date'}).get_text(strip=True)
            date = date.split(",")[1].strip()
            # Menggunakan fungsi kustom untuk mengonversi format tanggal
            date_obj = custom_strptime_detik(date)
            summary = article.find('p').string.strip()

            # Menambahkan data ke list
            data.append({"url": link, "title": judul, "date": date_obj, "summary": summary})

    # Membuat DataFrame dari list data
    df = pd.DataFrame(data)

    # Menyimpan DataFrame ke dalam file CSV (opsional)
    df.to_csv(f"{query}_headlines.csv", index=False, encoding='utf-8')

    return df

In [4]:
# Contoh penggunaan fungsi:
query = "kota+tasikmalaya"
total_pages = 20
df = scrape_detik_headlines(query, total_pages)
df.head()

,url,title,date,summary
0,https://www.detik.com/jabar/berita/d-6956616/k...,Kisah Asti Daur Ulang Sampah hingga Karyanya S...,2023-10-01 08:00:00,Asti Gustiasih (63) tak menjadikan keterbatasa...
1,https://www.detik.com/jabar/hukum-dan-kriminal...,Jejak Kriminal Bule AS yang Tusuk Mati Mertua ...,2023-09-29 16:30:00,"Arthur Leigh Welohr (35), WNA asal Amerika Ser..."
2,https://news.detik.com/berita/d-6956617/viral-...,Viral Foto Siswa SMP di Tasik Injak Kepala Tem...,2023-09-29 16:30:00,"Heboh video bullying siswa SMP di Cilacap, Jat..."
3,https://www.detik.com/jabar/berita/d-6954906/v...,Viral Foto Siswa SMP di Tasik Injak Kepala Tem...,2023-09-28 18:00:00,Warga Tasikmalaya dihebohkan dengan beredarnya...
4,https://www.detik.com/jabar/hukum-dan-kriminal...,Polisi Ungkap Aplikasi Walla Pemicu Pelajar Ba...,2023-09-28 14:30:00,Kasus pelajar Bandung jadi korban sodomi 2 pri...


In [5]:
df.to_excel("data/berita_detik.xlsx", index=False)

#### Pikiran Rakyat Headlines

In [6]:
def custom_strptime_pr(date_str):
    # Mengonversi bulan ke angka
    bulan_dict = {
        "Januari": "01",
        "Februari": "02",
        "Maret": "03",
        "April": "04",
        "Mei": "05",
        "Juni": "06",
        "Juli": "07",
        "Agustus": "08",
        "September": "09",
        "Oktober": "10",
        "November": "11",
        "Desember": "12"
    }
    for bulan, angka in bulan_dict.items():
        date_str = date_str.replace(bulan, angka)

    # Mengonversi ke format datetime
    date_time_obj = datetime.strptime(date_str, "%d %m %Y, %H:%M WIB")
    return date_time_obj

In [7]:
from urllib.parse import urlparse, parse_qs, urlunparse
def scrape_article_content_pr(article_url):
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    webpage = requests.get(article_url, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Mencari elemen konten berita yang sesuai
    article_content = soup.find('article', class_='read__content').find_all('p')

    # Mengambil teks dari elemen konten
    content_text = []
    for paragraph in article_content:
        content_text.append(paragraph.get_text(strip=True))

    # Menggabungkan semua teks konten
    article_text = ' '.join(content_text)
    print(article_text)

    # Mencari elemen paging untuk mengecek jumlah halaman
    paging_element = soup.find('div', class_='paging')
    if paging_element:
        paging_links = paging_element.find_all('a', class_='paging__link')
        num_pages = len(paging_links)
    else:
        num_pages = 1  # Jika tidak ada elemen paging, maka hanya ada satu halaman
    
    # Parse URL artikel
    parsed_url = urlparse(article_url)
    query_params = parse_qs(parsed_url.query)

    # Cek apakah ada halaman berikutnya dengan mengubah parameter page
    # dan mengambil kontennya jika ada
    page = 1
    while page <= num_pages:
        print(article_url)
        query_params['page'] = [str(page)]
        updated_url = urlunparse((
            parsed_url.scheme,
            parsed_url.netloc,
            parsed_url.path,
            parsed_url.params,
            '&'.join([f'{key}={value[0]}' for key, value in query_params.items()]),
            parsed_url.fragment
        ))
        print(updated_url)
        next_page_response = requests.get(updated_url, headers=HEADERS)
        if next_page_response.status_code == 200:
            next_page_soup = BeautifulSoup(next_page_response.content, "html.parser")
            next_article_content = next_page_soup.find('article', class_='read__content').find_all('p')
            if next_article_content:
                # Mengambil teks dari elemen konten halaman berikutnya
                next_content_text = [paragraph.get_text(strip=True) for paragraph in next_article_content]
                article_text += ' '.join(next_content_text)
                page += 1
            else:
                break
        else:
            break

    return article_text


In [8]:
def scrape_pr_headlines(total_pages=4):
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # Buat list untuk menyimpan data
    data = []

    base_url = f"https://www.pikiran-rakyat.com/search/?q=kota+tasikmalaya&page={{}}#gsc.tab=0&gsc.q=kota%20tasikmalaya&gsc.page=1"

    for page in range(1, total_pages + 1):
        URL = base_url.format(page)
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "lxml")
        articles = soup.find_all('div', attrs={'class': 'latest__item'})
        
        for article in articles:
            link = article.find('a')['href']
            judul = article.find('a', attrs={'class': 'latest__link'}).string.strip()
            date_str = article.find('date', attrs={'class': 'latest__date'}).get_text(strip=True)
            # Menggunakan fungsi custom_strptime_pr untuk mengkonversi tanggal
            date = custom_strptime_pr(date_str)

            # Menggunakan fungsi scrape_article_content untuk mengambil konten berita
            article_content = scrape_article_content_pr(link)
            
            # Menambahkan data ke list
            data.append({"url": link, "title": judul, "date": date, "content": article_content})

    # Membuat DataFrame dari list data
    df = pd.DataFrame(data)
    return df

In [9]:
# Contoh penggunaan fungsi:
total_pages = 10
df_pr = scrape_pr_headlines(total_pages)
df_pr.head()

 PIKIRAN RAKYAT–Matahari Mega MallKotaTasikmalayamengalamikebakaranpada Jumat, 9 Juni 2023 siang. Momenkebakaranini beredar luas di media sosial, salah satunya dibagikan di akun Instagram @infojawabarat. Dalam video yang beredar di akun Instagram tersebut, pengunjung dan pegawai yang bekerja diMatahari Mega Mallberhamburan menyelamatkan diri. Terlihat asap tebal dan kobaran api menyelimuti swalayan besar diTasikmalayaitu. Berdasarkan informasi yang beredar,kebakarandiMatahari Mega MallTasikmalayaterjadi sekira pukul 13.57 WIB. Adapun titikkebakarandisebut terjadi di area gudang lantai dua mall tersebut. Petugas pemadamkebakaranlangsung mendatangi lokasi dan berupaya memadamkan kobaran api. Butuh waktu cukup lama untuk memadamkan kobaran api yang besar tersebut. Baca Juga:Polemik Ekspor Pasir di Indonesia, PBB Justru Serukan Aksi Lindungi Lautan Dunia Pihak kepolisian dari Polsek Cihideung PolrestaTasikmalayasudah datang untuk mengamankan lokasi. Aparat langsung mencari tahu penyebabkeb

,url,title,date,content
0,https://www.pikiran-rakyat.com/jawa-barat/pr-0...,"Matahari Mega Mall Kota Tasikmalaya Kebakaran,...",2023-06-09 16:07:00,PIKIRAN RAKYAT–Matahari Mega MallKotaTasikmal...
1,https://www.pikiran-rakyat.com/jawa-barat/pr-0...,Polisi Tangkap Pria yang Mencabuli Keponakanny...,2023-06-02 20:00:00,PIKIRAN RAKYAT- Kepolisian Resor KotaTasikmal...
2,https://www.pikiran-rakyat.com/jawa-barat/pr-0...,Isi Rekaman Mediasi Siswa SMA Tasikmalaya deng...,2023-05-22 16:36:00,PIKIRAN RAKYAT -Komisi Perlindungan Anak Indon...
3,https://www.pikiran-rakyat.com/jawa-barat/pr-0...,"Jalan Rusak, Warga Keluhkan Genangan Air di Pa...",2023-05-16 18:11:00,PIKIRAN RAKYAT- Kondisi sejumlah ruasjalandi ...
4,https://www.pikiran-rakyat.com/jawa-barat/pr-0...,Kota Tasikmalaya Masuk 10 Besar Pencapaian Vak...,2023-05-15 07:00:00,PIKIRANRAKYAT- Angka cakupan pencapaian pelak...


In [23]:
df_pr.shape

(200, 4)

In [25]:
df_pr.to_excel("data/berita_pr.xlsx", index=False)

## Text Cleaning

In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/superapp-
[nltk_data]     research/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
import re, string, unicodedata  #modul regular expression
import nltk
from nltk import word_tokenize, sent_tokenize  #Paket ini membagi teks input menjadi kata-kata.,                                  
from nltk.corpus import stopwords

In [28]:
#preprocessing
def removeStopword(str):
    stop_words = set(stopwords.words('indonesian'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)
#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str
def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
       
    return str
def preprocessing(str):
    str = removeSentence(str)
    str = cleaning(str)
    str = removeStopword(str)
    
    return str

In [29]:
#test the code
sentences = ["dimana lokasi kuliner tasikmalaya yang murah","alamat tasik dimana sih, yang enak","s"]
for st in sentences:
    r = preprocessing(st)
    print(r)

dimana lokasi kuliner tasikmalaya murah
alamat tasik dimana sih enak



In [30]:
# data dari detik.com
txt_detik = df['summary'].apply(preprocessing)

# Save the preprocessed data to a new CSV file
txt_detik.to_csv('data/detik-clean-data.csv', index=False)


In [31]:
txt_pr = df_pr['content'].apply(preprocessing)

In [32]:
# Fungsi untuk menghapus frasa 'pikiran rakyat' dan 'pikiranrakyat'
def hapus_frasa(text):
    text = text.replace('pikiran rakyat', '').replace('pikiranrakyat', '')
    return text

In [33]:
# Menghapus frasa 'pikiran rakyat' dan 'pikiranrakyat' dari setiap teks dalam daftar
txt_pr_clean = txt_pr.apply(hapus_frasa)

In [34]:
txt_pr

0      pikiran rakyatmatahari mega mallkotatasikmalay...
1      pikiran rakyat kepolisian resor kotatasikmalay...
2      pikiran rakyat komisi perlindungan anak indone...
3      pikiran rakyat kondisi ruasjalandi wilayah kot...
4      pikiranrakyat angka cakupan pencapaian pelaksa...
                             ...                        
195    pikiran rakyat penetapan lokasi penlok jalur j...
196    pikiran rakyat badan penanggulangan bencana da...
197    pikiran rakyat selasa september positifcovid k...
198    pikiran rakyat kelompokberandalan bermotor gen...
199    pikiran rakyat mengantisipasi penyelewengan pe...
Name: content, Length: 200, dtype: object

In [35]:
txt_pr_clean

0      matahari mega mallkotatasikmalayamengalamikeba...
1       kepolisian resor kotatasikmalayaberhasil mena...
2       komisi perlindungan anak indonesia daerah kpa...
3       kondisi ruasjalandi wilayah kotatasikmalayasu...
4       angka cakupan pencapaian pelaksanaanvaksinpol...
                             ...                        
195     penetapan lokasi penlok jalur jalantol kota t...
196     badan penanggulangan bencana daerah bpbd kota...
197     selasa september positifcovid kotatasikmalaya...
198     kelompokberandalan bermotor geng motor berula...
199     mengantisipasi penyelewengan penyaluran bantu...
Name: content, Length: 200, dtype: object

In [36]:
txt_pr_clean.to_csv('data/pr-clean-data.csv', index=False)